## LLM (Large Language Model)
- LLMs are advanced AI models designed to understand and generate human-like text based on vast amounts of data.

Que son los llm y para que sirven?
## LLM (Large Language Model)
Un LLM multimodal es un modelo de lenguaje avanzado que puede entender, generar y razonar no solo con texto, sino también con otros tipos de datos como imágenes, audio o video. A diferencia de los modelos puramente textuales, un LLM multimodal combina información de diferentes fuentes para dar respuestas más completas y contextualizadas. Por ejemplo, puede analizar una imagen y describirla con palabras, interpretar texto dentro de ella o responder preguntas sobre lo que muestra.


In [3]:
from openai import OpenAI
import requests 
import os 
import base64
from pathlib import Path
import io
from PIL import Image
import fitz  # PyMuPDF - Agregar esta línea



def load_document(file_path: str) -> dict:
    """
    🎯 Función que carga documentos PDF o PNG y los convierte para LLM multimodal
    
    Args:
        file_path (str): Ruta al archivo PDF o PNG
        
    Returns:
        dict: Información del documento procesado
    """
    
    file_path = Path(file_path)
    
    if not file_path.exists():
        raise FileNotFoundError(f"❌ Archivo no encontrado: {file_path}")
    
    file_extension = file_path.suffix.lower()
    
    print(f"📄 Cargando documento: {file_path.name}")
    print(f"📋 Tipo de archivo: {file_extension}")
    
    if file_extension == '.pdf':
        return load_pdf_document(file_path)
    elif file_extension in ['.png', '.jpg', '.jpeg']:
        return load_image_document(file_path)
    else:
        raise ValueError(f"❌ Formato no soportado: {file_extension}")


def load_pdf_document(file_path: Path) -> dict:
    """
    📄 Carga y procesa documentos PDF
    
    Args:
        file_path (Path): Ruta al archivo PDF
        
    Returns:
        dict: PDF procesado como imágenes en base64
    """
    
    try:
        # Abrir el PDF
        pdf_document = fitz.open(file_path)
        total_pages = len(pdf_document)
        
        print(f"📖 PDF con {total_pages} páginas")
        
        pages_data = []
        
        for page_num in range(total_pages):
            print(f"   🔄 Procesando página {page_num + 1}/{total_pages}")
            
            # Obtener la página
            page = pdf_document[page_num]
            
            # Convertir página a imagen
            pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # 2x zoom para mejor calidad
            img_data = pix.tobytes("png")
            
            # Convertir a base64
            base64_image = base64.b64encode(img_data).decode('utf-8')
            
            pages_data.append({
                "page_number": page_num + 1,
                "base64_image": base64_image,
                "format": "png"
            })
        
        pdf_document.close()
        
        return {
            "file_name": file_path.name,
            "file_type": "pdf",
            "total_pages": total_pages,
            "pages": pages_data,
            "status": "success"
        }
        
    except Exception as e:
        return {
            "file_name": file_path.name,
            "file_type": "pdf", 
            "error": str(e),
            "status": "error"
        }


def load_image_document(file_path: Path) -> dict:
    """
    🖼️ Carga y procesa documentos de imagen
    
    Args:
        file_path (Path): Ruta al archivo de imagen
        
    Returns:
        dict: Imagen procesada en base64
    """
    
    try:
        # Abrir y procesar la imagen
        with Image.open(file_path) as img:
            # Convertir a RGB si es necesario
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Redimensionar si es muy grande (opcional)
            max_size = (1024, 1024)
            img.thumbnail(max_size, Image.Resampling.LANCZOS)
            
            # Convertir a bytes
            img_buffer = io.BytesIO()
            img.save(img_buffer, format='PNG')
            img_data = img_buffer.getvalue()
            
            # Convertir a base64
            base64_image = base64.b64encode(img_data).decode('utf-8')
            
            return {
                "file_name": file_path.name,
                "file_type": "image",
                "format": "png",
                "base64_image": base64_image,
                "dimensions": img.size,
                "status": "success"
            }
            
    except Exception as e:
        return {
            "file_name": file_path.name,
            "file_type": "image",
            "error": str(e),
            "status": "error"
        }


def convert_base64(file_path: str) -> str:
    """
    🔄 Función simplificada que convierte archivo a base64
    
    Args:
        file_path (str): Ruta al archivo
        
    Returns:
        str: Archivo en formato base64
    """
    
    try:
        document_data = load_document(file_path)
        
        if document_data["status"] == "error":
            raise Exception(document_data["error"])
        
        if document_data["file_type"] == "pdf":
            # Para PDF, devolver la primera página
            return document_data["pages"][0]["base64_image"]
        else:
            # Para imágenes
            return document_data["base64_image"]
            
    except Exception as e:
        print(f"❌ Error al convertir a base64: {str(e)}")
        return ""


# 🧪 FUNCIÓN DE PRUEBA
def test_document_loading():
    """
    🧪 Función para probar la carga de documentos
    """
    
    print("🧪 PROBANDO CARGA DE DOCUMENTOS")
    print("=" * 40)
    
    # Archivos de ejemplo (ajusta las rutas según tus archivos)
    test_files = [
        "comprobantes/mp_youtube_pago.pdf",
        "comprobantes/mp_pago_imagen.png", 
    ]
    
    for file_path in test_files:
        if os.path.exists(file_path):
            print(f"\n📄 Probando: {file_path}")
            try:
                result = load_document(file_path)
                
                if result["status"] == "success":
                    print(f"✅ Cargado exitosamente")
                    if result["file_type"] == "pdf":
                        print(f"   📖 Páginas: {result['total_pages']}")
                    else:
                        print(f"   📏 Dimensiones: {result['dimensions']}")
                else:
                    print(f"❌ Error: {result['error']}")
                    
            except Exception as e:
                print(f"❌ Error: {str(e)}")
        else:
            print(f"⚠️ Archivo no encontrado: {file_path}")

# Ejecutar pruebas (descomenta si quieres probar)
test_document_loading()



🧪 PROBANDO CARGA DE DOCUMENTOS

📄 Probando: comprobantes/mp_youtube_pago.pdf
📄 Cargando documento: mp_youtube_pago.pdf
📋 Tipo de archivo: .pdf
📖 PDF con 2 páginas
   🔄 Procesando página 1/2
   🔄 Procesando página 2/2
✅ Cargado exitosamente
   📖 Páginas: 2

📄 Probando: comprobantes/mp_pago_imagen.png
📄 Cargando documento: mp_pago_imagen.png
📋 Tipo de archivo: .png
✅ Cargado exitosamente
   📏 Dimensiones: (559, 726)


### Definicion de costos

In [4]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

MODEL_COSTS = {
    "gpt-4o": {
        "input": 0.0025,   # $2.50 por 1M tokens = $0.0025 por 1K ✅ Confirmado
        "output": 0.01,    # $10.00 por 1M tokens = $0.01 por 1K ✅ Confirmado
        "cached_input": 0.00125,  # 50% descuento con cache
        "vision": "Incluido en tokens estándar",  # ~700 tokens por imagen 1024x1024
        "quality": "Excelente",
        "speed": "Rápido (77.4 tokens/segundo)",
        "context": "128K tokens",
        "max_output": "16.4K tokens"
    },
    "gpt-4o-mini": {
        "input": 0.00015,  # $0.15 por 1M tokens ✅ Confirmado
        "output": 0.0006,  # $0.60 por 1M tokens ✅ Confirmado
        "cached_input": 0.000075,  # 50% descuento con cache
        "vision": "Incluido en tokens estándar",
        "quality": "Muy bueno (82% MMLU)",
        "speed": "Muy rápido",
        "context": "128K tokens",
        "max_output": "16K tokens"
    },
    
    # 🆕 NUEVOS MODELOS GEMINI 2.5
    "gemini-2.5-pro": {
        "input": 0.00125,  # $1.25 por 1M tokens (≤200K contexto)
        "input_large": 0.0025,  # $2.50 por 1M tokens (>200K contexto)
        "output": 0.01,    # $10.00 por 1M tokens (≤200K contexto)
        "output_large": 0.015,  # $15.00 por 1M tokens (>200K contexto)
        "vision": "Incluido",  # Sin costo adicional
        "audio": 0.001,    # $1.00 por 1M tokens audio
        "cache": 0.000125,  # Cache muy económico
        "quality": "Excelente - Modelo frontier",
        "speed": "Rápido",
        "context": "2M tokens",
        "batch_discount": "50%"  # Descuento para procesamiento batch
    },
    "gemini-2.5-flash": {
        "input": 0.0003,   # $0.30 por 1M tokens texto/imagen/video
        "output": 0.0025,  # $2.50 por 1M tokens
        "audio_input": 0.001,  # $1.00 por 1M tokens audio
        "vision": "Incluido",
        "cache": 0.00003,  # Cache ultra económico
        "quality": "Muy bueno - Modelo de razonamiento híbrido",
        "speed": "Muy rápido",
        "context": "1M tokens",
        "batch_discount": "50%"
    },
    "gemini-2.5-flash-lite": {
        "input": 0.0001,   # $0.10 por 1M tokens ⚡ MÁS BARATO
        "output": 0.0004,  # $0.40 por 1M tokens
        "audio_input": 0.0003,  # $0.30 por 1M tokens audio
        "vision": "Incluido",
        "cache": 0.000025,
        "quality": "Bueno - Optimizado para escala",
        "speed": "Ultra rápido",
        "context": "Contexto moderado",
        "batch_discount": "50%"
    },
    
    # 🆕 GEMINI 2.0 (Versión anterior pero aún disponible)
    "gemini-2.0-flash": {
        "input": 0.0001,   # $0.10 por 1M tokens
        "output": 0.0004,  # $0.40 por 1M tokens
        "audio_input": 0.0007,  # $0.70 por 1M tokens audio
        "vision": "Incluido",
        "cache": 0.000025,
        "quality": "Bueno - Era de agentes",
        "speed": "Muy rápido",
        "context": "1M tokens",
        "batch_discount": "50%"
    }
}


def calcular_costo(usage_metadata: dict, modelo: str = "gpt-4o-mini", context_size: int = 0) -> dict:
    """
    📊 Calcula el costo de uso de tokens para diferentes modelos
    
    Args:
        usage_metadata: Diccionario con input_tokens, output_tokens, total_tokens
        modelo: Nombre del modelo usado
        context_size: Tamaño del contexto (para modelos Gemini con precios variables)
    
    Returns:
        dict: Información detallada de tokens y costos
    """

    if modelo not in MODEL_COSTS:
        raise ValueError(f"❌ Modelo '{modelo}' no encontrado en MODEL_COSTS")

    precios = MODEL_COSTS[modelo]
    input_tokens = usage_metadata.get('input_tokens', 0)
    output_tokens = usage_metadata.get('output_tokens', 0)

    # 🔍 Determinar precio de input (algunos modelos tienen precios variables)
    if 'input_large' in precios and context_size > 200_000:
        precio_input = precios['input_large']
    elif 'cached_input' in usage_metadata and usage_metadata.get('cached_input', 0) > 0:
        # Si hay tokens cacheados, calcular separadamente
        precio_input = precios.get('cached_input', precios['input'])
        input_tokens = usage_metadata.get('cached_input', 0)
    else:
        precio_input = precios['input']

    # 🔍 Determinar precio de output
    if 'output_large' in precios and context_size > 200_000:
        precio_output = precios['output_large']
    else:
        precio_output = precios['output']

    # 💰 Calcular costos (precios ya están por 1K tokens)
    costo_input = (input_tokens / 1_000) * precio_input
    costo_output = (output_tokens / 1_000) * precio_output
    costo_total = costo_input + costo_output

    return {
        "modelo": modelo,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "total_tokens": usage_metadata.get('total_tokens', input_tokens + output_tokens),
        "costo_input_usd": round(costo_input, 6),
        "costo_output_usd": round(costo_output, 6),
        "costo_total_usd": round(costo_total, 6),
        "precio_input_por_1k": precio_input,
        "precio_output_por_1k": precio_output,
        "calidad": precios.get('quality', 'N/A'),
        "velocidad": precios.get('speed', 'N/A'),
    }


def ejecutar_llm(
    prompt_text: str,
    document: dict,
    model: str = "gpt-4o-mini",
    model_provider: str = "openai",
    system_message: str = "Eres un experto en análisis de documentos financieros.",
    verbose: bool = True
) -> dict:
    """
    🤖 Ejecuta un LLM multimodal con imagen y retorna respuesta + costos
    
    Args:
        prompt_text: Texto del prompt para analizar la imagen
        document: Diccionario con 'base64_image' del documento
        model: Nombre del modelo a usar
        model_provider: Proveedor del modelo (openai, google-genai, etc.)
        system_message: Mensaje de sistema para contextualizar
        verbose: Si True, imprime información de costos
    
    Returns:
        dict: {
            "respuesta": contenido de la respuesta,
            "costos": información de tokens y costos,
            "metadata": metadata completa de la respuesta
        }
    """

    # Inicializar el modelo
    llm = init_chat_model(model=model, model_provider=model_provider)

    # Construir mensajes
    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=[
            {
                "type": "text",
                "text": prompt_text
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{document['base64_image']}"
                }
            }
        ])
    ]

    # Invocar el modelo
    response = llm.invoke(messages)

    return response


def mostrar_resultado_costos(model, costos):
     """ Funcion que permite mostrar los costos por pantalla"""
     # Imprimir información si verbose=True
     print("\n" + "="*50)
     print(f"📊 ANÁLISIS DE COSTO - {model}")
     print("="*50)
     print(f"🔹 Tokens de entrada:  {costos['input_tokens']:,}")
     print(f"🔹 Tokens de salida:   {costos['output_tokens']:,}")
     print(f"🔹 Total tokens:       {costos['total_tokens']:,}")
     print(f"\n💰 COSTOS:")
     print(f"🔹 Entrada:  ${costos['costo_input_usd']:.6f}")
     print(f"🔹 Salida:   ${costos['costo_output_usd']:.6f}")
     print(f"🔹 TOTAL:    ${costos['costo_total_usd']:.6f}")
     print(f"\n⚙️  MODELO:")
     print(f"🔹 Calidad:    {costos['calidad']}")
     print(f"🔹 Velocidad:  {costos['velocidad']}")
     print("="*50 + "\n")
    


In [5]:
result = load_document("comprobantes/mp_pago_imagen.png")
print(result)

📄 Cargando documento: mp_pago_imagen.png
📋 Tipo de archivo: .png
{'file_name': 'mp_pago_imagen.png', 'file_type': 'image', 'format': 'png', 'base64_image': '...', 'dimensions': (559, 726), 'status': 'success'}


In [6]:
prompt = "Extrae la siguiente información en formato JSON: fecha, monto, destinatario, concepto"
resultado = ejecutar_llm(
    prompt_text=prompt,
    document={"base64_image": result["base64_image"]},  # ✅ Diccionario correcto
    model="gemini-2.5-flash-lite",
    model_provider="google-genai",
    verbose=True
)

# Calcular costos
costos = calcular_costo(resultado.usage_metadata, modelo="gpt-4o-mini")

mostrar_resultado_costos("gpt-4o-mini", costos)

   

print("✅ Extracción exitosa")
print(f"📄 Respuesta: {resultado.content}")

E0000 00:00:1760891541.753285  300308 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



📊 ANÁLISIS DE COSTO - gpt-4o-mini
🔹 Tokens de entrada:  285
🔹 Tokens de salida:   62
🔹 Total tokens:       347

💰 COSTOS:
🔹 Entrada:  $0.000043
🔹 Salida:   $0.000037
🔹 TOTAL:    $0.000080

⚙️  MODELO:
🔹 Calidad:    Muy bueno (82% MMLU)
🔹 Velocidad:  Muy rápido

✅ Extracción exitosa
📄 Respuesta: ```json
{
  "fecha": "18 de octubre de 2025",
  "monto": "$ 48.909",
  "destinatario": "Susana Armeya",
  "concepto": "Varios"
}
```


In [7]:
def test_modelos(file_path="comprobantes/mp_pago_imagen.png"):
    """
    🧪 Prueba simple para usar los modelos con análisis de costos
    """
    
    print("🧪 PROBANDO EXTRACCIÓN CON ANÁLISIS DE COSTOS")
    print("=" * 50)
    
    if not os.path.exists(file_path):
        print(f"❌ Archivo no encontrado: {file_path}")
        return
    
    # Cargar documento
    result = load_document(file_path)
    if result["status"] != "success":
        print(f"❌ Error cargando documento: {result.get('error')}")
        return
    
    # Preparar documento para LLM
    if result["file_type"] == "pdf":
        document_for_llm = {"base64_image": result["pages"][0]["base64_image"]}
    else:
        document_for_llm = {"base64_image": result["base64_image"]}
    
    # Prompt de extracción
    prompt = "Extrae la siguiente información en formato JSON: fecha, monto, destinatario, concepto"
    
    # Probar con diferentes modelos
    modelos_test = [
        {"model": "gpt-4o-mini", "provider": "openai"},
        {"model": "gemini-2.5-flash-lite", "provider": "google-genai"},
        {"model": "gemini-2.5-flash", "provider": "google-genai"}
    ]
    
    resultados_comparacion = []  # 📊 Para comparar al final
    
    for config in modelos_test:
        print(f"\n🤖 Probando: {config['model']}")
        try:
            resultado = ejecutar_llm(
                prompt_text=prompt,
                document=document_for_llm,
                model=config["model"],
                model_provider=config["provider"],
                verbose=False  # 🔧 Cambiado a False para evitar duplicar outputs
            )

            # 🔧 CORRECCIÓN PRINCIPAL - Usar el modelo correcto
            costos = calcular_costo(resultado.usage_metadata, modelo=config["model"])
            
            # Mostrar costos del modelo actual
            mostrar_resultado_costos(config["model"], costos)
            
            print("✅ Extracción exitosa")
            print(f"📄 Respuesta: {resultado.content[:100]}...")
            
            # Guardar para comparación final
            resultados_comparacion.append({
                "modelo": config["model"],
                "costo": costos['costo_total_usd'],
                "tokens": costos['total_tokens'],
                "calidad": costos['calidad'],
                "respuesta": resultado.content
            })
            
        except Exception as e:
            print(f"❌ Error con {config['model']}: {str(e)}")
    
    # 📊 MOSTRAR COMPARACIÓN FINAL
    if len(resultados_comparacion) > 1:
        print(f"\n{'='*60}")
        print("📊 RESUMEN COMPARATIVO DE MODELOS")
        print("="*60)
        
        # Ordenar por costo (más barato primero)
        resultados_comparacion.sort(key=lambda x: x['costo'])
        
        for i, r in enumerate(resultados_comparacion):
            emoji = "🏆" if i == 0 else "💰" if i == 1 else "📊"
            print(f"\n{emoji} {r['modelo']}:")
            print(f"   💵 Costo: ${r['costo']:.6f}")
            print(f"   📊 Tokens: {r['tokens']:,}")
            print(f"   ⭐ Calidad: {r['calidad']}")
        
        # Calcular ahorro
        mas_caro = max(resultados_comparacion, key=lambda x: x['costo'])
        mas_barato = min(resultados_comparacion, key=lambda x: x['costo'])
        
        if mas_caro['costo'] > mas_barato['costo']:
            ahorro_pct = ((mas_caro['costo'] - mas_barato['costo']) / mas_caro['costo']) * 100
            ahorro_usd = mas_caro['costo'] - mas_barato['costo']
            
            print(f"\n💡 AHORRO MÁXIMO:")
            print(f"   🎯 {ahorro_pct:.1f}% usando {mas_barato['modelo']} vs {mas_caro['modelo']}")
            print(f"   💰 ${ahorro_usd:.6f} por consulta")
            print(f"   📈 En 1000 consultas: ${ahorro_usd * 1000:.2f}")

# Ejecutar la prueba
test_modelos()

🧪 PROBANDO EXTRACCIÓN CON ANÁLISIS DE COSTOS
📄 Cargando documento: mp_pago_imagen.png
📋 Tipo de archivo: .png

🤖 Probando: gpt-4o-mini

📊 ANÁLISIS DE COSTO - gpt-4o-mini
🔹 Tokens de entrada:  25,539
🔹 Tokens de salida:   50
🔹 Total tokens:       25,589

💰 COSTOS:
🔹 Entrada:  $0.003831
🔹 Salida:   $0.000030
🔹 TOTAL:    $0.003861

⚙️  MODELO:
🔹 Calidad:    Muy bueno (82% MMLU)
🔹 Velocidad:  Muy rápido

✅ Extracción exitosa
📄 Respuesta: ```json
{
  "fecha": "18 de octubre de 2025",
  "monto": 48.909,
  "destinatario": "Susana Armeya",
...

🤖 Probando: gemini-2.5-flash-lite


E0000 00:00:1760891553.649769  300308 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



📊 ANÁLISIS DE COSTO - gemini-2.5-flash-lite
🔹 Tokens de entrada:  285
🔹 Tokens de salida:   61
🔹 Total tokens:       346

💰 COSTOS:
🔹 Entrada:  $0.000028
🔹 Salida:   $0.000024
🔹 TOTAL:    $0.000053

⚙️  MODELO:
🔹 Calidad:    Bueno - Optimizado para escala
🔹 Velocidad:  Ultra rápido

✅ Extracción exitosa
📄 Respuesta: ```json
{
  "fecha": "18 de octubre de 2025",
  "monto": "48.909",
  "destinatario": "Susana Armeya"...

🤖 Probando: gemini-2.5-flash


E0000 00:00:1760891555.742119  300308 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



📊 ANÁLISIS DE COSTO - gemini-2.5-flash
🔹 Tokens de entrada:  285
🔹 Tokens de salida:   1,061
🔹 Total tokens:       1,346

💰 COSTOS:
🔹 Entrada:  $0.000085
🔹 Salida:   $0.002652
🔹 TOTAL:    $0.002738

⚙️  MODELO:
🔹 Calidad:    Muy bueno - Modelo de razonamiento híbrido
🔹 Velocidad:  Muy rápido

✅ Extracción exitosa
📄 Respuesta: ```json
{
  "fecha": "Sábado, 18 de octubre de 2025 a las 11:36 hs",
  "monto": "$ 48.909",
  "desti...

📊 RESUMEN COMPARATIVO DE MODELOS

🏆 gemini-2.5-flash-lite:
   💵 Costo: $0.000053
   📊 Tokens: 346
   ⭐ Calidad: Bueno - Optimizado para escala

💰 gemini-2.5-flash:
   💵 Costo: $0.002738
   📊 Tokens: 1,346
   ⭐ Calidad: Muy bueno - Modelo de razonamiento híbrido

📊 gpt-4o-mini:
   💵 Costo: $0.003861
   📊 Tokens: 25,589
   ⭐ Calidad: Muy bueno (82% MMLU)

💡 AHORRO MÁXIMO:
   🎯 98.6% usando gemini-2.5-flash-lite vs gpt-4o-mini
   💰 $0.003808 por consulta
   📈 En 1000 consultas: $3.81
